In [4]:
import pandas as pd
import numpy as np
#import tensorflow as tf
#from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import robust_scale
#tf.keras.backend.clear_session()

In [5]:
from typing import List

def assign_split_col(df: pd.DataFrame, col: str, name_list: List[str], pat: str=None):
    df = df.copy()
    split_col = df[col].str.split(pat, expand=True)

    return df.assign(
        **dict(
            zip(name_list, [split_col.iloc[:, x] for x in range(split_col.shape[1])])
        )
    )

In [6]:
df = pd.read_csv("../../data/raw/data.csv")


In [7]:
edited_df = (
    df.pipe(
        assign_split_col,
        col="Unnamed: 0",
        name_list=["nth_chunk", "File", "Participant"],
        pat=".",
    )
    .loc[lambda x: ~x["Participant"].isna()]
    .sort_values(by="nth_chunk")
    .drop(columns=["Unnamed: 0", "nth_chunk", "File"])
    .assign(y = lambda x: x["y"].mask(lambda x: x>1, 0))

)

train, validation = train_test_split(
    edited_df, test_size=0.3, random_state=0, stratify=edited_df[["Participant", "y"]]
)

In [9]:
with_split_ids = (
    df.pipe(
        assign_split_col,
        col="Unnamed: 0",
        name_list=["nth_chunk", "File", "Participant"],
        pat=".",
    )
    .loc[lambda x: ~x["Participant"].isna()]
    .sort_values(by=["Participant", "nth_chunk"])
    .drop(columns=["Unnamed: 0"])

)

In [10]:
with_split_ids.to_csv("../../data/interim/with_split_ids.csv", index=False)

In [7]:
train.to_csv("../../data/interim/train.csv", index=False)

In [8]:
validation.to_csv("../../data/interim/validation.csv", index=False)

In [9]:
#Keep a seperate Validation set that's never been seen

In [ ]:
import pandas as pd
import stumpy
import numpy as np
from numba import cuda
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import itertools
from numba import cuda

orig_df = pd.read_csv("../../data/interim/train.csv")
df = orig_df.drop(columns=["y", "Participant"])

df_float = df.astype(float)

all_gpu_devices = [device.id for device in cuda.list_devices()]

m = 30
df_float_with_mean_mp = df_float.assign(mean_mp = lambda x: 
                                        x.apply(lambda y: 
                                                stumpy.gpu_stump(y, 
                                                                 m = m, 
                                                                 device_id=all_gpu_devices )[:, 0].mean(),
                                               axis=1))

df_float_with_mean_mp["y"] = orig_df["y"]

In [ ]:
import pandas as pd
import stumpy
import numpy as np
from numba import cuda
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from matplotlib.patches import Rectangle
import datetime as dt
import itertools
from numba import cuda

orig_df = pd.read_csv("../../data/interim/train.csv")
df = orig_df.drop(columns=["y", "Participant"])
df_float = df.astype(float)
all_gpu_devices = [device.id for device in cuda.list_devices()]

m = 30

mps = df_float.apply(lambda y: 
                     stumpy.gpu_stump(y, 
                                      m = m, 
                                      device_id=all_gpu_devices )[:, 0],
                     axis=1)

mps_df = pd.DataFrame(mps)

reshaped = mps_df.apply(lambda y: 
                     pd.Series(y.values[0]),
                     axis=1)

reshaped["y"] = orig_df["y"]

reshaped.to_csv("../../data/interim/transformed_to_mps.csv", index=False)

In [ ]:
mp_df = pd.read_csv("../../data/interim/transformed_to_mps.csv")

X = mp_df.drop(columns = ["y"])
y = mp_df["y"]

descs = X.apply(lambda x: x.describe(), axis=1)

X_modded = descs.drop(columns = ["count"])

X_modded_with_y = pd.concat([X_modded, y], axis=1)

X_modded_with_y.to_csv("../../data/interim/mps_descs.csv", index=False)